코딩도장 내용정리

### 1. 데코레이터 만들기
데코레이터는 함수를 수정하지 않은 상태에서 추가 기능을 구현할 때 사용.  

In [3]:
# 함수의 시간과 끝을 출력하고 싶을때 데코레이터를 사용하지 않은 방법
def hello():
    print('hello 함수 시작')
    print('hello')
    print('hello 함수 끝')
    
def world():
    print('world 함수 시작')
    print('world')
    print('world 함수 끝')
    
hello()
world()

hello 함수 시작
hello
hello 함수 끝
world 함수 시작
world
world 함수 끝


In [5]:
# 함수의 시작과 끝을 출력하는 데코레이터를 사용하는 방법
def trace(func):
    def wrapper():
        print(func.__name__, '함수 시작')
        func()
        print(func.__name__, '함수 끝')
    return wrapper

def hello():
    print('hello')
    
def world():
    print('world')
    
trace_hello = trace(hello)    # 데코레이터에 호출할 함수를 넣음
trace_hello()                 # 반환된 함수 호출
trace_world = trace(world)
trace_world()

hello 함수 시작
hello
hello 함수 끝
world 함수 시작
world
world 함수 끝


1. 데코레이터 trace는 호출할 함수를 매개변수로 받고 호출할 함수를 감싸는 함수 wrapper를 만든다.  
2. wrapper 함수에서는 함수의 시작과 끝을 알리는 문자열을 출력하도록 하고 그 사이에 매개변수로 받은 func를 호출.
    - 매개변수로 받은 함수의 원래 이름을 출력할 때는 `__name__`속성 활용
3. wrapper함수를 만든 후 trace함수에서 return을 사용하여 wrapper 함수 자체를 반환.
    - 즉 함수 안에서 함수를 만들고 반환하는 클로저
4. trace에 호출할 함수를 넣어 반환된 함수를 호출.

#### @로 데코레이터 사용하기
호출할 함수 위에 @데코레이터 형식으로 지정
```python
@decorator
def func():
    code
```

In [6]:
def trace(func):
    def wrapper():
        print(func.__name__, '함수 시작')
        func()
        print(func.__name__, '함수 끝')
    return wrapper

@trace      # @데코레이터
def hello():
    print('hello')
    
@trace
def world():
    print('world')
    
hello()
world()

hello 함수 시작
hello
hello 함수 끝
world 함수 시작
world
world 함수 끝


#### 데코레이터의 그림 표현
<img src="https://dojang.io/pluginfile.php/14011/mod_page/content/2/042001.png">

함수를 감싸는 형태로 구성되어 있어서 기존 함수를 수정하지 않으면서 추가 기능을 구현할 때 사용된다.  

#### 여러 개의 데코레이터 지정
```python
@decorator1
@decorator2
def func():
    code
```
데코레이터의 실행 순서는 위에서 아래 순..

In [8]:
def decorator1(func):
    def wrapper():
        print('decorator1')
        func()
    return wrapper

def decorator2(func):
    def wrapper():
        print('decorator2')
        func()
    return wrapper

# 여러 개의 데코레이터 지정
@decorator1
@decorator2
def hello():
    print('hello')
    
hello()

decorator1
decorator2
hello


In [10]:
# @을 사용하지 않는다면 다음과 같다
def hello():
    print('hello')
decorated_hello = decorator1(decorator2(hello))
decorated_hello()

decorator1
decorator2
hello


### 2. 매개변수와 반환값을 처리하는 데코레이터 만들기

매개변수와 반환값을 처리하는 데코레이터를 만들어 보자

In [11]:
def trace(func):
    def wrapper(a, b):
        r = func(a, b)
        print(f'{func.__name__}(a={a}, b={b}) -> {r}')
        return r
    return wrapper

@trace
def add(a, b):
    return a + b

print(add(10, 20))

add(a=10, b=20) -> 30
30


1. 먼저 안쪽 wrapper 함수의 매개변수를 호출할 함수의 매개변수와 똑같이 만들어준다.
2. wrapper 함수 안에서는 func를 호출하고 반환값을 변수에 저장한다.
3. func의 반환값을 return으로 반환한다.

#### 가변 인수 함수 데코레이터
매개변수(인수)가 고정되지 않은 함수 처리. wrapper 함수를 가변 인수 함수로 만들자.

In [12]:
def trace(func):
    def wrapper(*args, **kwargs):
        r = func(*args, **kwargs)
        print(f'{func.__name__,}(args={args}, kwargs={kwargs}) -> {r}')
        return r
    return wrapper

@trace
def get_max(*args):
    return max(args)

@trace
def get_min(**kwargs):
    return min(kwargs.values())

print(get_max(10, 20))
print(get_min(x=10, y=20, z=30))

('get_max',)(args=(10, 20), kwargs={}) -> 20
20
('get_min',)(args=(), kwargs={'x': 10, 'y': 20, 'z': 30}) -> 10
10


#### 메서드에 데코레이터 사용하기
메서드에 데코레이터를 사용할 때는 self를 주의하자!  
데코레이터를 만들 때도 wrapper함수의 첫번째 매개변수를 self로 지정해야 한다.(class method는 cls).  
func을 호출할 때도 self와 매개변수를 그대로 넣어야 한다.

In [13]:
def trace(func):
    def wrapper(self, a, b):
        r = func(self, a, b)
        print(f'{func.__name__}(a={a}, b={b}) -> {r}')
        return r
    return wrapper

class Calc:
    @trace
    def add(self, a, b):
        return a + b
    
c = Calc()
print(c.add(10, 20))

add(a=10, b=20) -> 30
30


### 3. 매개변수가 있는 데코레이터 만들기
매개변수가 있는 데코레이터를 만들어보자.  
이런 방식의 데코레이터는 값을 지정해서 동작을 바꿀 수 있다.

In [14]:
# 함수의 반환값이 특정 수의 배수인지 확인하는 데코레이터
def is_multiple(x):                # 데코레이터가 사용할 매개변수 지정
    def real_decorator(func):      # 호출할 함수를 매개변수로 받음.
        def wrapper(a, b):         # 호출할 함수 매개변수와 맞춰줌.
            r = func(a, b)
            if r % x == 0:
                print(f'{func.__name__}의 반환값은 {x}의 배수입니다.')
            else:
                print(f'{func.__name__}의 반환값은 {x}의 배수가 아닙니다.')
            return r
        return wrapper
    return real_decorator

@is_multiple(3)   # 매개변수가 있는 데코레이터.
def add(a, b):
    return a + b

print(add(10, 20))
print(add(2, 5))

add의 반환값은 3의 배수입니다.
30
add의 반환값은 3의 배수가 아닙니다.
7


1. 매개변수가 있는 데코레이터를 만들 때는 함수를 3중으로 만들어야 한다.
2. 먼저 가장 바깥에 있는 함수에 데코레이터가 사용할 매개변수 x를 지정.
3. 그 안에 실제 데코레이터 역할을 하는 real_decorator 생성.
    - 호출할 함수를 배개변수로 받는다.
4. real_decorator 함수 안에서 wrapper 함수를 만들어준다.

#### 매개변수가 있는 데코레이터 여러 개 지정

In [15]:
@is_multiple(3)
@is_multiple(7)
def add(a, b):
    return a + b
 
add(10, 20)

add의 반환값은 7의 배수가 아닙니다.
wrapper의 반환값은 3의 배수입니다.


30

#### @을 사용하지 않았을 때 동작

In [16]:
def add(a, b):
    return a + b
decorated_add = is_multiple(3)(is_multiple(7)(add))
decorated_add(10, 20)

add의 반환값은 7의 배수가 아닙니다.
wrapper의 반환값은 3의 배수입니다.


30

#### wrapper가 이름으로 나오네??
데코레이터를 여러 개 사용하면 데코레이터에서 반환된 wrapper 함수가 다른 데코레이터로 들어가기 때문에 함수의 `__name__`을 출력하면 wrapper가 나온다.  
원래 함수의 이름을 출력하고 싶다면 functools 모듈의 wraps 데코레이터를 사용해야 한다.  
@functools.wraps에 func를 넣은 뒤 wrapper 함수 위에 지정해준다.

In [17]:
import functools

def is_multiple(x):
    def real_decorator(func):
        @functools.wraps(func)   # @functools.wraps에 func를 넣은 뒤 wrapper 함수 위에 지정.
        def wrapper(a, b):
            r = func(a, b)
            if r % x == 0:
                print(f'{func.__name__}의 반환값은 {x}의 배수입니다.')
            else:
                print(f'{func.__name__}의 반환값은 {x}의 배수가 아닙니다.')
            return r
        return wrapper
    return real_decorator

@is_multiple(3)
@is_multiple(7)
def add(a, b):
    return a + b

add(10, 20)

add의 반환값은 7의 배수가 아닙니다.
add의 반환값은 3의 배수입니다.


30

원래 함수의 정보를 유지시켜주기 때문에 디버깅을 할 때 유용.  
데코레이터를 만들 때는 @functools.wraps를 사용하는 것이 좋다!

### 4. 클래스로 데코레이터 만들기
클래스로 데코레이터를 만들어 보자.  
클래스를 활용할 때는 인스턴스를 함수처럼 호출하게 해주는 `__call__`메서드를 구현해야 한다.  

In [18]:
# 함수의 시작과 끝을 출력하는 데코레이터
class Trace:
    def __init__(self, func):
        self.func = func
        
    def __call__(self):
        print(self.func.__name__, '함수 시작')
        self.func()
        print(self.func.__name__, '함수 끝')
        
@Trace
def hello():
    print('hello')
    
hello()

hello 함수 시작
hello
hello 함수 끝


1. 먼저 `__init__` 메서드를 만들고 호출할 함수를 초깃값으로 받는다.  
2. 그리고 매개변수로 받은 함수를 속성으로 저장.(self.func = func)  
3. 그리고 인스턴스를 호출할 수 있도록 `__call__`메서드를 만든다.  
    - `__call__`메서드에서는 함수의 시작과 끝을 알리는 문자열을 출력
    - 그 사이에 속성 func에 저장된 함수를 호출한다.
4. 클로져 형태의 데코레이터와 같이 사용한다.

#### 참고
클래스로 만든 데코레이터는 @을 지정하지 않고, 데코레이터의 반환값을 호출하는 방식으로도 사용할 수 있다.

In [19]:
def hello():
    print('hello')
    
trace_hello = Trace(hello)   # 데코레이터에 호출할 함수를 넣어서 인스턴스 생성
trace_hello()                # 인스턴스를 호출. __call__메서드가 호출됨.

hello 함수 시작
hello
hello 함수 끝


클래스에 `__call__` 메서드를 정의했으므로 인스턴스를 함수처럼 괄호()를 붙여서 호출할 수 있다.

### 5. 클래스로 매개변수와 반환값을 처리하는 데코레이터 만들기

In [21]:
class Trace:
    def __init__(self, func):
        self.func = func
        
    def __call__(self, *args, **kwargs):
        r = self.func(*args, **kwargs)
        print(f'{self.func.__name__}(args={args}, kwargs={kwargs}) -> {r}')
        return r

@Trace
def add(a, b):
    return a + b

print(add(10, 20))
print(add(a=10, b=20))

add(args=(10, 20), kwargs={}) -> 30
30
add(args=(), kwargs={'a': 10, 'b': 20}) -> 30
30


`__call__`메서드에 매개변수를 지정하고, self.func에 매개변수를 넣어서 호출한 뒤 반환값을 반환.  
매개변수를 `*args`, `**kwargs`로 지정했으므로 self.func에 넣을 때는 언패킹하여 넣어줌.

고정된 매개변수를 사용할 때는 `def __call__(self, a, b):`처럼 하면 된다.

#### 매개변수가 있는 데코레이터 만들기

In [23]:
class IsMultiple:
    def __init__(self, x):
        self.x = x
        
    def __call__(self, func):
        def wrapper(a, b):
            r = func(a, b)
            if r % self.x == 0:
                print(f'{func.__name__}의 반환값은 {self.x}의 배수입니다.')
            else:
                print(f'{func.__name__}의 반환값은 {self.x}의 배수가 아닙니다.')
            return r
        return wrapper
    
@IsMultiple(3)
def add(a, b):
    return a + b

print(add(10, 20))
print(add(2, 5))

add의 반환값은 3의 배수입니다.
30
add의 반환값은 3의 배수가 아닙니다.
7


1. `__init__`메서드에서 데코레이터가 사용할 매개변수를 초깃값으로 받음
    - 매개변수를 `__call__`메서드에서 사용할 수 있도록 속성에 저장
    - 이전에는 매개변수로 호출할 함수를 인자로 받았었다!!
2. `__call__` 메서드에서 호출할 함수를 매개변수로 받음.
3. `__call__` 메서드 안에서 wrapper함수를 만들어준다.
    - wrapper 함수의 매개변수는 호출할 함수의 매개변수와 똑같이.(가변인수도 가능)

#### 정리
데코레이터는 기존 함수를 수정하지 않으면서 추가 기능을 구현할 때 사용!  
디버깅, 함수 성능 측정, 함수 실행 전에 데이터 확인 등에 활용된다.

#### 문제 1. 데코레이터로 매개변수의 자료형 검사하기

In [24]:
def type_check(type1, type2):
    def real_decorator(func):
        def wrapper(a, b):
            if type(a)==type1 and type(b)==type2:
                r = func(a, b)
                return r
            else:
                raise RuntimeError('자료형이 올바르지 않습니다.')
        return wrapper
    return real_decorator

@type_check(int, int)
def add(a, b):
    return a + b

print(add(10, 20))
print(add('hello', 'world'))

30


RuntimeError: 자료형이 올바르지 않습니다.

매개변수를 받는 데코레이터는 함수를 삼중으로!!

#### 문제 2. HTML 태그 데코레이터 만들기
표준 입력으로 HTML 태그 이름 두 개가 입력. 함수의 반환값을 HTML 태그로 감싸는 데코레이터를 만들어보자..

In [26]:
def html_tag(tag_name):
    def real_decorator(func):
        def wrapper():
            r = f'<{tag_name}>{func()}</{tag_name}>'
            return r
        return wrapper
    return real_decorator

a, b = input().split()

@html_tag(a)
@html_tag(b)
def hello():
    return 'Hello, world!'

print(hello())

b i
<b><i>Hello, world!</i></b>


데코레이터는 이해 좀 된듯...